In [2]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_excel('BA_Stock news with sentiment.xlsx')

In [17]:
df = df.dropna(subset=['content'])

In [19]:
df = df.drop(columns=['publisher', 'url', 'title'])
df

,date,content,negative,neutral,positive
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497
...,...,...,...,...,...
9038,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062
9039,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060
9040,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281
9041,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393


## Загрузка Fine_Tune модели для NER

In [43]:
from transformers import BertTokenizer, BertForTokenClassification, AutoConfig, pipeline
from safetensors.torch import load_file
import os


In [51]:
from transformers import pipeline, BertTokenizer, BertForTokenClassification, AutoConfig
from safetensors.torch import load_file

# Загрузка конфигурации и модели
config_path = "config.json"
config = AutoConfig.from_pretrained(config_path)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Загрузка модели с конфигурацией
model = BertForTokenClassification.from_pretrained(
    None,
    state_dict=load_file("model.safetensors"),
    config=config
)

# Создание pipeline для NER
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Тестовое предложение
example = "North Airlines, Google and Microsoft are leading tech companies. IBM is also a major player."

# Предсказание NER
ner_results = nlp(example)

# Фильтрация результатов, оставляя только B-ORG и I-ORG
org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]

# Вывод отфильтрованных результатов
print("Результаты NER (только B-ORG и I-ORG):")
for result in org_results:
    print(result)


Результаты NER (только B-ORG и I-ORG):
{'entity': 'B-ORG', 'score': 0.9873624, 'index': 1, 'word': 'north', 'start': None, 'end': None}
{'entity': 'I-ORG', 'score': 0.97545075, 'index': 2, 'word': 'airlines', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.9943715, 'index': 4, 'word': 'google', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.99384284, 'index': 6, 'word': 'microsoft', 'start': None, 'end': None}
{'entity': 'B-ORG', 'score': 0.99315506, 'index': 12, 'word': 'ibm', 'start': None, 'end': None}


In [61]:
def find_entity(example):
    ner_results = nlp(example)
    org_results = [result for result in ner_results if result['entity'] in ['B-ORG', 'I-ORG']]
    
    entities = []
    stack = []
    # Пройдемся по списку, избегая выхода за пределы индекса
    for i in range(len(org_results)):
        entity = org_results[i]['entity']
        word = org_results[i]['word']
        
        if entity == 'B-ORG':
            # Если начало новой организации, очистим стек
            stack = [word]
            # Если следующий элемент - тоже 'B-ORG', добавим текущий стек в 'entities'
            if i + 1 < len(org_results) and org_results[i + 1]['entity'] == 'B-ORG':
                entities.append(' '.join(stack))
        elif entity == 'I-ORG':
            # Если продолжение организации, добавим в стек
            stack.append(word)

        # Если конец списка или следующий элемент - новое начало, добавляем текущий стек в 'entities'
        if i + 1 >= len(org_results) or org_results[i + 1]['entity'] == 'B-ORG':
            entities.append(' '.join(stack))
    return entities

In [65]:
df['entities'] = np.nan
df['entities'] = df['content'].apply(find_entity)

In [66]:
df

,date,content,negative,neutral,positive,entities
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,[boeing]
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"[u, u, ., ., s, s, . justice department]"
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,[boeing]
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"[japan airlines, airbus, airbus, boeing]"
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"[st, st, ##ife, ##ife, ##l, ##l, boeing]"
...,...,...,...,...,...,...
9038,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"[& ##insky, usd, usd, ##c, ##c, boeing, boeing..."
9039,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"[globe news ##wire, levi & ko ##rs ##insky, us..."
9040,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"[globe news, hagen, hagen, ##s berman sob ##ol..."
9041,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"[globe news ##wire, po, po, ##mer, ##mer, ##an..."


In [69]:
df['content'][4]

"Bert Subin, Aerospace Analyst at Stifel, gives his expectations for Boeing's earnings report."

In [70]:
df['entities'][4]

['st', 'st', '##ife', '##ife', '##l', '##l', 'boeing']

In [71]:
df.to_csv('BA_with_entities.csv', index=False) 

In [76]:
df

,date,content,negative,neutral,positive,entities
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,[boeing]
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"[u, u, ., ., s, s, . justice department]"
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,[boeing]
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"[japan airlines, airbus, airbus, boeing]"
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"[st, st, ##ife, ##ife, ##l, ##l, boeing]"
...,...,...,...,...,...,...
9038,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"[& ##insky, usd, usd, ##c, ##c, boeing, boeing..."
9039,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"[globe news ##wire, levi & ko ##rs ##insky, us..."
9040,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"[globe news, hagen, hagen, ##s berman sob ##ol..."
9041,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"[globe news ##wire, po, po, ##mer, ##mer, ##an..."


In [117]:
df_tickers = pd.read_csv('List_tickers.csv', sep=';')
df_tickers

,id,ticker,company,exchange,exchangeFullName,isPrimary,primaryTicker,sector,industryGroup,industry,subIndustry,currency,country,ipoDate,delistedDate,trackNews
0,1,MAPT4.SA,Cemepe Investimentos S.A.,BVMF,Sao Paolo Exchange,NaN,NaN,40.0,NaN,NaN,NaN,BRL,BR,27.01.2000,NaN,0
1,2,CPLE3.SA,Companhia Paranaense de Energia - COPEL,BVMF,Sao Paolo Exchange,NaN,NaN,55.0,NaN,NaN,NaN,BRL,BR,03.01.2000,NaN,0
2,3,CPLE6.SA,Companhia Paranaense de Energia - COPEL,BVMF,Sao Paolo Exchange,NaN,NaN,55.0,NaN,NaN,NaN,BRL,BR,03.01.2000,NaN,0
3,4,POM.TO,PolyMet Mining Corp.,XTSE,Toronto Exchange,NaN,NaN,15.0,1510.0,151040.0,15104020.0,CAD,US,08.08.1996,NaN,0
4,5,PTM.TO,Platinum Group Metals Ltd.,XTSE,Toronto Exchange,NaN,NaN,15.0,1510.0,151040.0,15104020.0,CAD,CA,18.06.1998,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52015,53336,ELC,"Entergy Louisiana, LLC COLLATERAL TR MT",NYSE,New York Stock Exchange,NaN,NaN,55.0,NaN,NaN,NaN,USD,US,13.10.2010,NaN,0
52016,53337,WRLGF,West Red Lake Gold Mines Ltd.,PNK,Other OTC,NaN,NaN,15.0,1510.0,151040.0,15104030.0,USD,CA,11.06.2010,NaN,0
52017,53338,TUXS,Tuxis Corporation,PNK,Other OTC,NaN,NaN,NaN,NaN,NaN,NaN,USD,US,08.11.1996,NaN,0
52018,53339,GHTI,G-H-3 International Inc.,PNK,Other OTC,NaN,NaN,NaN,NaN,NaN,NaN,USD,US,05.06.2023,NaN,0


In [118]:
df_tickers = df_tickers[df_tickers['trackNews'] == 1]
df_tickers = df_tickers[['ticker', 'company']]
df_tickers = df_tickers.reset_index(drop=True)
df_tickers

,ticker,company
0,CE,Celanese Corporation
1,J,Jacobs Engineering Group Inc.
2,DD,"DuPont de Nemours, Inc."
3,ABT,Abbott Laboratories
4,LDOS,"Leidos Holdings, Inc."
5,AXP,American Express Company
6,DIS,The Walt Disney Company
7,T,AT&T Inc.
8,XOM,Exxon Mobil Corporation
9,APD,"Air Products and Chemicals, Inc."


In [119]:
len(df_tickers)

58

In [146]:
df_stox = pd.read_csv('STOXX600.csv', sep=';')
df_stox = df_stox[['code', 'name']]
df_stox = df_stox.reset_index(drop=True)
df_stox.head(5)

,code,name
0,1COV.DE,COVESTRO AG
1,A2A.MI,A2A
2,AAK.ST,AAK
3,AAL.L,ANGLO AMERICAN PLC
4,AALB.AS,AALBERTS NV


In [143]:
df_sp500 = pd.read_csv('SP500.csv', sep=';')
df_sp500 = df_sp500[['code', 'name']]
df_sp500 = df_sp500.reset_index(drop=True)
df_sp500.head(5)


,code,name
0,ABT,Abbott Laboratories
1,ABBV,AbbVie
2,ACN,Accenture
3,ADBE,Adobe Inc.
4,AMD,Advanced Micro Devices Inc.


In [144]:
df_nsdq100 = pd.read_csv('NSDQ100.csv', sep=';')
df_nsdq100 = df_nsdq100[['code', 'name']]
df_nsdq100 = df_nsdq100.reset_index(drop=True)
df_nsdq100.head(5)

,code,name
0,AAPL,Apple Inc.
1,ADBE,Adobe Inc.
2,ADI,Analog Devices Inc.
3,ADP,Automatic Data Processing Inc.
4,ADSK,Autodesk Inc.


In [145]:
df_hahg = pd.read_csv('HangSeng.csv', sep=';')
df_hahg = df_hahg[['code', 'name']]
df_hahg = df_hahg.reset_index(drop=True)
df_hahg.head(5)

,code,name
0,0011.HK,Hang Seng Bank Limited
1,0388.HK,Hong Kong Exchanges and Clearing Limited
2,0939.HK,China Construction Bank Corporation
3,1299.HK,AIA Group Limited
4,1398.HK,Industrial & Commercial Bank of China Ltd.


In [152]:
data_tickers = pd.concat([df_sp500, df_nsdq100, df_hahg, df_stox])
data_tickers = data_tickers.drop_duplicates(subset=['code'])
data_tickers = data_tickers.reset_index(drop=True)
data_tickers.head()

,code,name
0,ABT,Abbott Laboratories
1,ABBV,AbbVie
2,ACN,Accenture
3,ADBE,Adobe Inc.
4,AMD,Advanced Micro Devices Inc.


In [168]:
df['tickers'] = np.nan
df

,date,content,negative,neutral,positive,entities,tickers
0,2024-04-24,Boeing has been mired in a safety crisis since...,0.981807,0.017569,0.000624,[boeing],NaN
1,2024-04-24,Families of the victims of two fatal Boeing 73...,0.653259,0.345061,0.001681,"[u, u, ., ., s, s, . justice department]",NaN
2,2024-04-24,"Merle Meyers, who left Boeing last year after ...",0.000208,0.999556,0.000236,[boeing],NaN
3,2024-04-24,Japan Airlines will continue using both Airbus...,0.000074,0.999808,0.000118,"[japan airlines, airbus, airbus, boeing]",NaN
4,2024-04-24,"Bert Subin, Aerospace Analyst at Stifel, gives...",0.000277,0.000226,0.999497,"[st, st, ##ife, ##ife, ##l, ##l, boeing]",NaN
...,...,...,...,...,...,...,...
9038,2016-02-26,"NEW YORK, NY--(Marketwired - February 26, 2016...",0.000131,0.999807,0.000062,"[& ##insky, usd, usd, ##c, ##c, boeing, boeing...",NaN
9039,2016-02-24,"NEW YORK, Feb. 24, 2016 (GLOBE NEWSWIRE) -- ...",0.000141,0.999800,0.000060,"[globe news ##wire, levi & ko ##rs ##insky, us...",NaN
9040,2016-02-24,"SAN FRANCISCO, Feb. 24, 2016 (GLOBE NEWSWIRE...",0.857394,0.141325,0.001281,"[globe news, hagen, hagen, ##s berman sob ##ol...",NaN
9041,2016-02-22,"NEW YORK, Feb. 22, 2016 (GLOBE NEWSWIRE) -- ...",0.000274,0.999333,0.000393,"[globe news ##wire, po, po, ##mer, ##mer, ##an...",NaN


In [153]:
data_tickers['name'] = data_tickers['name'].str.lower()
data_tickers.head(5)

,code,name
0,ABT,abbott laboratories
1,ABBV,abbvie
2,ACN,accenture
3,ADBE,adobe inc.
4,AMD,advanced micro devices inc.


In [154]:
data_tickers_name = data_tickers['name'].to_list()
data_tickers_name

['abbott laboratories',
 'abbvie',
 'accenture',
 'adobe inc.',
 'advanced micro devices inc.',
 'advance auto parts',
 'aes corp',
 'aflac',
 'agilent technologies, inc.',
 'air products & chemicals',
 'akamai technologies',
 'alaska air group',
 'albemarle corporation',
 'alexandria real estate equities',
 'align technology inc.',
 'allegion',
 'alliant energy',
 'allstate corp',
 'alphabet inc. (class a)',
 'alphabet inc. (class c)',
 'altria group',
 'amazon inc.',
 'amcor',
 'ameren corp',
 'american airlines group',
 'american electric power co inc.',
 'american express',
 'american international group',
 'american tower',
 'american water works',
 'ameriprise financial',
 'cencora, inc. common stock',
 'ametek',
 'amgen inc.',
 'amphenol corp',
 'analog devices inc.',
 'ansys inc.',
 'elevance health, inc. common stock',
 'aon',
 'a. o. smith',
 'apa corporation',
 'apple inc.',
 'applied materials inc.',
 'aptiv',
 'adm',
 'arista networks',
 'arthur j. gallagher & co.',
 'assu

In [134]:
ticker_boeing = data_tickers[data_tickers['code'] == 'BA']
ticker_boeing

,code,name
10,BA,boeing company


In [136]:
japan_airlines = df['entities'][3] 
japan_airlines

['japan airlines', 'airbus', 'airbus', 'boeing']

In [137]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [140]:
ticker_boeing['name']

10    boeing company
Name: name, dtype: object

In [141]:
a = process.extract(str(ticker_boeing['name']), japan_airlines, limit=2)
# Параметр limit по умолчанию имеет значение 5
print(a)

[('boeing', 90), ('japan airlines', 39)]


## Pipeline

In [156]:
data_tickers.head(5)

,code,name
0,ABT,abbott laboratories
1,ABBV,abbvie
2,ACN,accenture
3,ADBE,adobe inc.
4,AMD,advanced micro devices inc.


In [155]:
a = process.extract(str(ticker_boeing['name']), japan_airlines, limit=5)
# Параметр limit по умолчанию имеет значение 5
print(a)

[('boeing', 90), ('japan airlines', 39), ('airbus', 15), ('airbus', 15)]


In [198]:
df = df.reset_index(drop=True)

In [199]:
for index in range(len(df)):
    # Выполняем fuzzy matching для извлечения потенциальных компаний
    extraction_results = process.extract(str(df['entities'][index]), data_tickers_name, limit=5)

    # Фильтруем компании со score >= 75
    matching_companies = [company for company, score in extraction_results if score >= 87]

    if matching_companies:
        # Получаем коды компаний, соответствующие matching_companies
        matching_codes = [data_tickers[data_tickers['name'] == company]['code'].values[0] for company in matching_companies]

        codes_str = ", ".join(matching_codes)

        df['tickers'].iloc[index] = codes_str

/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_14089/2010053881.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tickers'].iloc[index] = codes_str
/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_14089/2010053881.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tickers'].iloc[index] = codes_str
/var/folders/yj/nhkf3nv505d5kh0dsk42f3ym0000gn/T/ipykernel_14089/2010053881.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [201]:
df['tickers'].value_counts()

tickers
BA                         1492
UAL                          96
LUV                          38
AIR.PA                       22
AMR                          22
                           ... 
CBH                           1
DG, GM                        1
AXP, AMT, BAC, BDX, CMA       1
SHEL, SHELL.AS                1
DOV                           1
Name: count, Length: 81, dtype: int64